In [3]:
import tensorflow as tf
import random
import numpy as np
import cv2
import glob
from tqdm import tqdm
from PIL import Image
from pathlib import Path
import pickle
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:
training_data = []
DATADIR = "C:/Users/User/Desktop/ML/Clothes"
CATEGORIES = ["Notags", "Tags"]
IMG_SIZE = 256
def create_training_data():
    for category in CATEGORIES:  # notags and tags categories

        path = os.path.join(DATADIR,category)  # create path to notags and tags
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=nptag 1=tag

        for img in tqdm(os.listdir(path)):  # iterate over each image per notags and tags
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize image size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # pass errors
                pass

create_training_data() 

print(len(training_data))

In [ ]:
import random
#Shuffle data
random.shuffle(training_data)

In [ ]:
X = []
y = []
#Split to features and labels
for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)    

In [ ]:
#Normalize RGB from (0,255) to (0,1)
X = X/255.0

In [ ]:
#Store features array on disk
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
#Store labels array on disk
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [2]:
#Load features and labels without a need to extract and clean data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [13]:
#Name for tensorboard logs AlexNEt implementation 74-77% valaccuracy
IMG_SIZE = 256
NAME = "Tags-notags-clothes-AlexNEt-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
#Name for tensorboard logs My CNN 80-83% valaccuracy
NAME = "Tags-notags-clothes-CNN-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
IMG_SIZE = 256
# Model Architecture 
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [18]:
#Model compilation
model.compile(loss="binary_crossentropy",
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
activation_14 (Activation)   (None, 254, 254, 16)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 125, 125, 32)      4640      
_________________________________________________________________
activation_15 (Activation)   (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 60, 60, 64)       

In [20]:
#Training with 15 epochs and 20% validation split. Logs of loss and accuracy are sent to tensorboard
epochs = 15
model.fit(X, y, batch_size=32, validation_split=0.2, epochs=epochs,verbose=1, callbacks=[tensorboard])

Epoch 1/15
142/142 [==============================] - 19s 133ms/step - loss: 0.1106 - accuracy: 0.9582 - val_loss: 0.7572 - val_accuracy: 0.8046
Epoch 2/15
142/142 [==============================] - 4s 29ms/step - loss: 0.0993 - accuracy: 0.9665 - val_loss: 0.8158 - val_accuracy: 0.8143
Epoch 3/15
142/142 [==============================] - 4s 29ms/step - loss: 0.0992 - accuracy: 0.9674 - val_loss: 0.9623 - val_accuracy: 0.8389
Epoch 4/15
142/142 [==============================] - 4s 30ms/step - loss: 0.0862 - accuracy: 0.9714 - val_loss: 0.9293 - val_accuracy: 0.8275
Epoch 5/15
142/142 [==============================] - 4s 30ms/step - loss: 0.0859 - accuracy: 0.9729 - val_loss: 0.8903 - val_accuracy: 0.8231
Epoch 6/15
142/142 [==============================] - 4s 30ms/step - loss: 0.0865 - accuracy: 0.9690 - val_loss: 1.0076 - val_accuracy: 0.8169
Epoch 7/15
142/142 [==============================] - 4s 30ms/step - loss: 0.0746 - accuracy: 0.9753 - val_loss: 0.9272 - val_accuracy: 0.83